### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
movies.describe()

,movie_id,movie,genre
count,32242,32242,32010
unique,32242,32191,1194
top,4876546,Home (2016),Drama
freq,1,4,3684


In [3]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [5]:
# more work
reviews.describe()

,rating
count,739864.000000
mean,7.299851
std,1.856707
min,0.000000
25%,6.000000
50%,7.000000
75%,9.000000
max,10.000000


In [7]:
# and still more
movies['genre'].nunique()

1194

In [8]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0068646,10,1381620027
1,1,0113277,10,1379466669
2,2,0422720,8,1412178746
3,2,0454876,8,1394818630
4,2,0790636,7,1389963947


In [9]:
reviews['user_id'].nunique()

55280

In [10]:
reviews.isnull().mean()

user_id      0.0
movie_id     0.0
rating       0.0
timestamp    0.0
dtype: float64

In [ ]:
# and even more - open additional cells as necessary

In [11]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 32242, 
'The number of ratings in the dataset': 739864,
'The number of different genres': 1194,
'The number of unique users in the dataset': 55280,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.3,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [20]:
# cell for work
extract_year = lambda val: val[-5:-1] if val[-1] == ")" else np.nan

movies['year'] = movies['movie'].apply(extract_year)

def create_century_dummies(val):
    if val[:2] == century:
        return 1
    else:
        return 0
        
for century in ['18', '19', '20']:
    movies[century + "00's"] = movies['year'].apply(create_century_dummies)
    
movies.head()

,movie_id,movie,genre,year,1800's,1900's,2000's,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,1896


In [22]:
# and another

#extract genres
genres = []

for value in movies.genre:
    try:
        genres.extend(value.split('|'))
    except:
        pass
genres = set(genres)
genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [23]:
# and a few more below, which you can use as necessary
def split_genres(val):
    try:
        if val.find(ind_genre) >= 1:
            return 1
        else:
            return 0
    except:
        pass

for ind_genre in genres:
    movies[ind_genre] = movies['genre'].apply(split_genres)
    
movies.head()

,movie_id,movie,genre,year,1800's,1900's,2000's,date,Drama,News,...,Family,Horror,Documentary,Musical,Crime,Romance,Sport,History,Thriller,Action
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1894,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1895,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1896,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,1895,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,1896,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
reviews['timestamp'].head()

0    1381620027
1    1379466669
2    1412178746
3    1394818630
4    1389963947
Name: timestamp, dtype: object

In [28]:
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [29]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0068646,10,1381620027,2013-10-12 23:20:27
1,1,0113277,10,1379466669,2013-09-18 01:11:09
2,2,0422720,8,1412178746,2014-10-01 15:52:26
3,2,0454876,8,1394818630,2014-03-14 17:37:10
4,2,0790636,7,1389963947,2014-01-17 13:05:47


In [ ]:
reviews_new, movies_new = t.show_clean_dataframes()